In [ ]:
from ase import Atoms
from ase.build import bulk
from ase.io import write

# Example: Create a bulk copper structure (FCC, lattice a=3.61 Å)
atoms = bulk('Ar', 'fcc', a=5.26, cubic=True)

# (Optional) Create a 4x4x4 supercell
supercell = atoms.repeat((4, 4, 4))

# Write to a LAMMPS data file
write('Ar.data', supercell, format='lammps-data')